In [1]:
import numpy as np
import pandas as pd

AP_data = pd.DataFrame(columns=["match_id","hero_id","player_slot","radiant_win_res"])
#113 heros
Match_data_columns = range(1,114)
Match_data = pd.DataFrame(columns=Match_data_columns)

print(Match_data)

In [2]:
%fs ls /FileStore/tables/

In [3]:
df_data = pd.read_csv("/dbfs/FileStore/tables/match_hero_pick_all.csv", header=0)
print(df_data)

In [4]:
count = 1
Match_Result = pd.DataFrame(columns=["radiant_win_res"])
for index, row in df_data.iterrows():
  match_id = row['match_id']
  hero_id = row['hero_id']
  player_slot = row['player_slot']
  radiant_win = row['radiant_win']
  if radiant_win == True:
      radiant_win_res = True
  else:
      radiant_win_res = False
  Match_Result.loc[match_id] = radiant_win_res
  if count % 10 == 1:
      Match_data.loc[match_id] = np.zeros(113)
  if count % 10 < 6:
      Match_data.loc[match_id][hero_id] = 1
  else:
      Match_data.loc[match_id][hero_id] = -1
  #Match_data.loc[match_id]['label'] = radiant_win_res
  count += 1

In [5]:
Match_data['radiant_win'] = Match_Result.astype(int)

In [6]:
Match_data.describe()

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

categoricalColumns = []
#categoricalColumns = map(str, range(1,227))
#categoricalColumns.remove('24')
#categoricalColumns.remove('108')
#categoricalColumns.remove('113')
#categoricalColumns.remove('137')
#categoricalColumns.remove('221')
#categoricalColumns.remove('226')
#print(categoricalColumns)

stages = []
for categoricalCol in categoricalColumns:
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [stringIndexer, encoder]

In [8]:
label_stringIdx = StringIndexer(inputCol = "radiant_win", outputCol = "label")
stages += [label_stringIdx]

In [9]:
numericCols = map(str, range(1,114))
assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [10]:
spark_df = sqlContext.createDataFrame(Match_data)

pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(spark_df)
dataset = pipelineModel.transform(spark_df)

display(dataset)

In [11]:
(trainingData, testData) = dataset.randomSplit([0.91, 0.09], seed = 800)
trainingData.cache()

In [12]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [13]:
display(lrModel, trainingData, "ROC")

In [14]:
display(lrModel, testData, "ROC")

In [15]:
predictions = lrModel.transform(testData)

In [16]:
display(predictions)

In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

In [18]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.05, 0.5, 1, 2])
             .addGrid(lr.elasticNetParam, [0, 0.5, 1])
             .addGrid(lr.maxIter, [10])
             .build())

In [19]:
# Create 10-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=10)
cvModel = cv.fit(trainingData)

In [20]:
predictions = cvModel.transform(testData)
evaluator.evaluate(predictions)

In [21]:
from pyspark.ml.classification import RandomForestClassifier

# Create an initial RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

# Train model with Training Data
rfModel = rf.fit(trainingData)
predictions = rfModel.transform(testData)

In [22]:
display(predictions)

In [23]:
print rf.explainParams()

In [24]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

In [25]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [4, 6, 8, 10])
             .addGrid(rf.maxBins, [20, 40, 60])
             .addGrid(rf.numTrees, [20, 40])
             .build())

In [26]:
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
cvModel = cv.fit(trainingData)

In [27]:
predictions = cvModel.transform(testData)
evaluator.evaluate(predictions)